In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [6]:
PROPHET_PATH = '../data/prophet/'
BLENDS_PATH = 'blends/'
RESULTS_PATH = 'results/'
BLEND_NUMBER = 'b1/'
assert BLEND_NUMBER[-1] == '/'
VERSION = 'v3f/'
assert VERSION[-1] == '/'
assert VERSION[-2] == 'f'

In [5]:
med_df = pd.read_feather('../data/median_selected.f'); med_df.head()

,Id,Visits
0,ff8c1aade3de,18.0
1,6a4de5b43179,18.0
2,559a1ab24535,18.0
3,8bd4a4dea5a3,18.0
4,fb31ab105824,18.0


In [29]:
proph_df = pd.read_csv(PROPHET_PATH+'submissions/'+'backup_'+BLEND_NUMBER[:-1]+VERSION[:-1]+'.csv')

In [30]:
proph_df.head()

,Id,Visits
0,11c45df91fcf,63
1,957e6a70d6e0,53
2,66e316f6315f,49
3,337a0d125696,44
4,e918d253b59e,42


In [37]:
len(pd.Series(list(set(proph_df.Id) & set(med_df.Id))))/len(ds)

4236.0

In [35]:
ds = pd.DataFrame(pd.date_range('1/1/2017', '3/1/2017'), columns=['ds'])

In [36]:
len(ds)

60

In [39]:
med_df_fin = med_df[~med_df.Id.isin(proph_df.Id)]

In [42]:
len(med_df_fin)+len(proph_df)

8426580

In [40]:
with open(PROPHET_PATH+'submissions/'+BLEND_NUMBER[:-1]+VERSION[:-1]+'.csv', 'a') as f:
    med_df_fin.to_csv(f, header=False, index=False)

In [9]:
sub = pd.read_csv(PROPHET_PATH+'submissions/'+BLEND_NUMBER[:-1]+VERSION[:-1]+'.csv')

In [11]:
sub.head()

,Id,Visits
0,11c45df91fcf,63.0
1,957e6a70d6e0,53.0
2,66e316f6315f,49.0
3,337a0d125696,44.0
4,e918d253b59e,42.0


In [18]:
sub.Id.value_counts() == 2

def73d0f6b7a     True
7c2eb5bbe002     True
bfa3c37226b0     True
3ff178f6d312     True
8e148ca34051     True
fcd06cd7683d     True
acf336f1ac67     True
e71886eb8503     True
e8d03d46e8c8     True
10808434663b     True
0c8acb4d6fdc     True
456d0d1f2811     True
e5c9d4bf41c6     True
daee588eafd8     True
dfdf65a28d8c     True
36ec2f56fd72     True
5ee31f36d20d     True
8b34840b473c     True
5a25dad65c5f     True
62c1ac31f666     True
8c64f91550cc     True
81e1ef37c0ef     True
f3c223888b78     True
2f3ba8d98eea     True
85ebd0fa388c     True
e7a0c56601c6     True
1509e54bee4a     True
aa67d6321f64     True
2b141bfd2f9c     True
7ac86b96c53a     True
                ...  
838b9c385fc2    False
291f082a0d2f    False
dc3919baa687    False
9cce36c81687    False
5a31b714688c    False
2a7ed1e7e1ad    False
a7c9acbdf075    False
367c3d2193ac    False
b8aaf17b1ad0    False
ad65566591b8    False
dfc4952c0eae    False
c1d8439b8c0e    False
72dfbe4625f7    False
eab3256705b7    False
ff7da19059

In [16]:
sub.Id.value_counts().value_counts()

1    8172420
2     254160
Name: Id, dtype: int64

In [14]:
sub[sub.Id == 'def73d0f6b7a']

,Id,Visits
2746339,def73d0f6b7a,30.0
4774639,def73d0f6b7a,26.0


In [19]:
sub[sub.Id == '85ebd0fa388c']

,Id,Visits
1018007,85ebd0fa388c,0.0
4112027,85ebd0fa388c,0.0


In [24]:
sub.head()

,Id,Visits
0,11c45df91fcf,63.0
1,957e6a70d6e0,53.0
2,66e316f6315f,49.0
3,337a0d125696,44.0
4,e918d253b59e,42.0


In [23]:
sub.loc[3643442-10:3643442+10]

,Id,Visits
3643432,28ba8e861773,0.0
3643433,84ccef257dc4,0.0
3643434,39b0670e90a7,0.0
3643435,05e95dc2a347,0.0
3643436,b3c33fcde8d6,0.0
3643437,fdf3867a9be9,0.0
3643438,d2347cee9c8c,0.0
3643439,291247344360,0.0
3643440,ff8c1aade3de,18.0
3643441,6a4de5b43179,18.0


In [27]:
sub.loc[3643442:].Id.value_counts()

37e540116655    1
1891c361624d    1
581e24e041d3    1
752fafce6ac6    1
e5b0f8f71880    1
cd24ab21eca9    1
e1bad5158bbc    1
e8db79254cb2    1
9d54d79a4be8    1
62c919787bd6    1
2f6c61597338    1
2ff4851c876e    1
fd4e27dc6a5c    1
00438395fd1d    1
d1c8cf552f14    1
a14e0789861c    1
fbb03e100ce9    1
da7410026a4a    1
566b71e655e7    1
b5b62af011db    1
07a54bfbebc9    1
0fa859fd0824    1
d733fd217a63    1
f428dafb8d43    1
d004ef01ce19    1
d717864edde9    1
6f6dd4d6d0a3    1
35bec8bcdef2    1
4f57821add9e    1
6a3878ba15f2    1
               ..
7095ffd7363f    1
3f0c43b82253    1
22ebbde8710d    1
34fa8034fbb0    1
ae99317e9e5e    1
d3e0bc0b31fa    1
f2c3fe8b3f2d    1
8b24e278f9a3    1
ba3fa39f23d4    1
f76e88d68a1a    1
af4543ad92ab    1
5a419ee701e9    1
5daec3515a07    1
6ae6ba3beab3    1
9826817f8f65    1
d658fa061427    1
5dadcb89fc95    1
bdf7a4e00e72    1
5481d64af44e    1
383177d4b4c7    1
c796f46a00f9    1
5f7f32009cf0    1
0903baf3c356    1
5a478fba14ce    1
65bfd9244a

In [28]:
sub.loc[:3643442].Id.value_counts()

2e91de4c32f5    1
a4e80b32582c    1
e9abd0a294aa    1
7acef4218f6f    1
714e944b74e4    1
e804a8d3e752    1
b705c6a444c5    1
3052fd981b34    1
20d5e585c6b8    1
a1d76e11e359    1
3d8068eb3495    1
b47086009617    1
037606a4ae48    1
79a6621929c3    1
a19e2b269849    1
3d1d90fd45ba    1
4768b35b317d    1
dc6e097a7130    1
efb977ce8399    1
5f2c404c44f1    1
fe02544d5a5f    1
18d749844c5c    1
4b46439af510    1
a9872abbd94a    1
b0c6f6d95318    1
455c676c80e9    1
5f17aa51805a    1
ad7f56099521    1
c3829bd5c28c    1
7e530c7b76d8    1
               ..
d708d1b68e66    1
f37119a5ba7a    1
cfb194f515ff    1
83af0b0e6df0    1
d785ea806bfa    1
03643627d7fb    1
91d533c69262    1
c98ba7629216    1
c38182c69560    1
2287e197a01c    1
5f279aca8783    1
2184329810e9    1
810706fb4eb9    1
2b6730a7ad67    1
838e63479422    1
c09d7692030c    1
23eb2b3fc8cd    1
ea0c5b3a92e4    1
c2672abde002    1
619e4db09ef8    1
70edc9822288    1
632fc4973d49    1
05f1178c69a3    1
affd8c6f86be    1
914547a254

In [41]:
(8703780- 8426581)/60

4619.983333333334

In [3]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'
VERSION = 'v3f/'

In [4]:
page_index = pd.read_feather(PROPHET_PATH+'page_index.f')
keydf = pd.read_csv('../data/key_1.csv')

In [7]:
tdf = page_index.set_index('page_index').head()

In [11]:
tdf.loc[0].Page

'2NE1_zh.wikipedia.org_all-access_spider'

In [13]:
PROPHET_PATH = '../data/prophet/'
BLENDS_PATH = 'blends/'
RESULTS_PATH = 'results/'
BLEND_NUMBER = 'b1/'
assert BLEND_NUMBER[-1] == '/'
VERSION = 'v3f/'
assert VERSION[-1] == '/'
assert VERSION[-2] == 'f'

In [15]:
blend_pages = pd.read_feather(PROPHET_PATH+BLENDS_PATH+BLEND_NUMBER
                              +VERSION[:-2]+'df.f')

In [22]:
blend_pages = blend_pages.values.flatten()

In [5]:
# January, 1st, 2017 up until March 1st, 2017.
ds = pd.DataFrame(pd.date_range('1/1/2017', '3/1/2017'), columns=['ds'])

In [6]:
# Submission Make

In [7]:
#page_index = page_index.loc[:10]

In [12]:
subf = open(PROPHET_PATH+'submissions/'+VERSION[:-1]+'.csv', 'w')
subf.write('Id,Visits\n')
for row in tqdm(page_index.iterrows()):
    # use row[1][0] to load model
    with open(PROPHET_PATH+VERSION+str(row[1][0])+'m.pk', 'rb') as pkf:
        m = pk.load(pkf)
    # use model + ds to get prediction
    df = m.predict(ds).loc[:,['ds','yhat']]
    df['yhat_org'] = df['yhat']
    df.loc[df['yhat'] < 0,['yhat']] = 0.0
    # use row[1][1] and apply to make key col
    df['Page'] = df.ds.apply(lambda x: row[1][1]+'_'+str(x.date()))
    df = df.merge(keydf, on='Page', how='left').loc[:,['Id','yhat']]
    df.to_csv(subf,header=False, index=False)
    # write it to csv
subf.close()

0it [00:00, ?it/s]

0 <class 'int'>


1it [00:05,  5.62s/it]

             Id       yhat
0  ff8c1aade3de  28.166355
1  6a4de5b43179  21.471587
2  559a1ab24535  30.126437
3  8bd4a4dea5a3  27.603581
4  fb31ab105824  26.497807
1 <class 'int'>


2it [00:09,  5.22s/it]

...

             Id       yhat
0  6a5c449e172f  38.112184
1  063ed5676a95  38.974936
2  c7c13c0627db  38.395549
3  47855a21fd90  39.271972
4  e7e075db26c7  39.006479
9 <class 'int'>


10it [00:44,  4.24s/it]

             Id       yhat
0  64f6d0b3ac3d  33.144321
1  eb62a2a7447c  36.284368
2  5be02129d490  35.899996
3  afb022c7fa63  38.164868
4  cb2845c9c7e6  37.577186
10 <class 'int'>


11it [00:50,  4.76s/it]

             Id       yhat
0  4353ba20ec1f  14.078595
1  771e411282da  13.657523
2  4a0644dc0f55  15.530862
3  c4dfec2e79f6  12.826155
4  fec71974900c  19.691935
